In [1]:
%cd /hdd/yuchen

/hdd/yuchen


In [2]:
from PIL import Image
import torch
from torch import nn, optim
import glob
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, BatchSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
import nltk, re, string, collections
import scipy 
import IProgress 
from ipywidgets import FloatProgress
from nilearn import connectome
from sklearn.preprocessing import Normalizer, OrdinalEncoder, OneHotEncoder, StandardScaler
from torch.nn import Linear, ReLU, Dropout
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling, BatchNorm, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests
import pandas as pd

%matplotlib inline

In [3]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.cuda.empty_cache()

In [4]:
featname = 'c'
use_stim = 'each'

subject= 'subj01'
topdir = '/hdd/yuchen/cs292/nsdfeat'

savedir = f'{topdir}/subjfeat/'
featdir = f'{topdir}/15dim/{featname}/'

nsd_expdesign = scipy.io.loadmat('/hdd/yuchen/cs292/nsd/nsddata/experiments/nsd/nsd_expdesign.mat')

# Note that most of them are 1-base index!
sharedix = nsd_expdesign['sharedix'] -1 
stims = np.load(f'/hdd/yuchen/cs292/mrifeat/{subject}/{subject}_stims.npy')

feats = []
tr_idx = np.zeros(len(stims))

ct = 0

for idx, s in tqdm(enumerate(stims)): 
    if s in sharedix:
        tr_idx[idx] = 0
    else:
        tr_idx[idx] = 1    
    if os.path.exists(f'{featdir}/{s:06}.npy'):
        feat = np.load(f'{featdir}/{s:06}.npy')
        feats.append(feat)
    else: 
        ct += 1

feats = np.stack(feats)    

feats_tr = feats[tr_idx==1]
feats_te = feats[tr_idx==0]

print(ct, 'stims not found')

0it [00:00, ?it/s]

0 stims not found


In [5]:
import glob

dir = '/hdd/yuchen/cs292/mrifeat/subj01/'
exclude_lst = ['lateral', 'midlateral', 'midparietal', 'midventral','early',
              'V1', 'V2', 'V3', 'V3A', 'V3B', 'V3CD', 'V4', 'V4t',
               'stims', 'stims.npy',
               'PFt',
              ]
# https://github.com/ReedOnePeck/MindDiffuser/blob/main/Feature%20decoding/Semantic_feature_decoding.py
include_lst = ['VO','PHC','MT','MST','LO','IPS','hV4', 'OFC', 'FFC', 'FOP', 'PF', 'V6', 'V7', 'V8', 'RSC','TO','OP','VMV','VVC']

lst = []
lst_all = []
for file_path in glob.glob(f"{dir}/*.npy"):
    area = file_path.split('/')[-1].split('_')[1]
    for key in include_lst:
        if key in area and area not in exclude_lst: 
            lst.append(area)
    lst_all.append(area)
            # pass
    # if area in include_lst:
    # lst.append(area)
lst = np.unique(np.array(lst))
lst

array(['FFC', 'FOP1', 'FOP2', 'FOP3', 'FOP4', 'FOP5', 'IPS0', 'IPS1',
       'IPS2', 'IPS3', 'IPS4', 'IPS5', 'LO1', 'LO2', 'LO3', 'MST', 'MT',
       'OFC', 'OP1', 'OP2-3', 'OP4', 'PF', 'PFcm', 'PFm', 'PFop', 'PHC1',
       'PHC2', 'RSC', 'TO1', 'TO2', 'V6', 'V6A', 'V7', 'V8', 'VMV1',
       'VMV2', 'VMV3', 'VO1', 'VO2', 'VVC', 'hV4', 'pOFC'], dtype='<U5')

In [6]:
np.unique(lst_all)

array(['1', '10d', '10pp', '10r', '10v', '11l', '13l', '2', '23c', '23d',
       '24dd', '24dv', '25', '31a', '31pd', '31pv', '33pr', '3a', '3b',
       '4', '43', '44', '45', '46', '47l', '47m', '47s', '52', '55b',
       '5L', '5m', '5mv', '6a', '6d', '6ma', '6mp', '6r', '6v', '7AL',
       '7Am', '7PC', '7PL', '7Pm', '7m', '8Ad', '8Av', '8BL', '8BM', '8C',
       '9-46d', '9a', '9m', '9p', 'A1', 'A4', 'A5', 'AAIC', 'AIP', 'AVI',
       'DVT', 'EC', 'FEF', 'FFC', 'FOP1', 'FOP2', 'FOP3', 'FOP4', 'FOP5',
       'FST', 'H', 'IFJa', 'IFJp', 'IFSa', 'IFSp', 'IP0', 'IP1', 'IP2',
       'IPS0', 'IPS1', 'IPS2', 'IPS3', 'IPS4', 'IPS5', 'Ig', 'LBelt',
       'LIPd', 'LIPv', 'LO1', 'LO2', 'LO3', 'MBelt', 'MI', 'MIP', 'MST',
       'MT', 'OFC', 'OP1', 'OP2-3', 'OP4', 'PBelt', 'PCV', 'PEF', 'PF',
       'PFcm', 'PFm', 'PFop', 'PFt', 'PGi', 'PGp', 'PGs', 'PH', 'PHA1',
       'PHA2', 'PHA3', 'PHC1', 'PHC2', 'PHT', 'PI', 'PIT', 'POS1', 'POS2',
       'PSL', 'PeEc', 'Pir', 'PoI1', 'PoI2', 'PreS', 'Pr

In [7]:
savedir = '/hdd/yuchen/cs292/mrifeat/subj01/'
# roi = 'V1'
ct = 0
roi = lst[0]

betas_tr = np.load(f'{savedir}/{subject}_{roi}_betas_tr.npy')
betas_te = np.load(f'{savedir}/{subject}_{roi}_betas_te.npy')
betas_tr_avg = np.load(f'{savedir}/{subject}_{roi}_betas_ave_tr.npy')
betas_te_avg = np.load(f'{savedir}/{subject}_{roi}_betas_ave_te.npy')

num_trials_tr, num_neurons = betas_tr.shape
num_trials_te_avg, _ = betas_te_avg.shape
roi_neuron = {roi: num_neurons}

for roi in lst[1:]:
    ct += 1
    print(ct,'/',len(lst)-1, end='\r')
    original_array = np.load(f'{savedir}/{subject}_{roi}_betas_tr.npy')
    roi_neuron[roi] = original_array.shape[1]
    betas_tr = np.hstack([betas_tr,original_array])
    original_array = np.load(f'{savedir}/{subject}_{roi}_betas_ave_te.npy')
    betas_te_avg = np.hstack([betas_te_avg,original_array])


In [8]:
stims = np.load(f'/hdd/yuchen/cs292/mrifeat/{subject}/{subject}_stims_ave.npy')

feats_avg = []
tr_idx = np.zeros(len(stims))

ct = 0

for idx, s in tqdm(enumerate(stims)): 
    if s in sharedix:
        tr_idx[idx] = 0
    else:
        tr_idx[idx] = 1    
    if os.path.exists(f'{featdir}/{s:06}.npy'):
        feat = np.load(f'{featdir}/{s:06}.npy')
        feats_avg.append(feat)
    else: 
        print(f'{s:06}.npy does not exist')
        feats_avg.append(np.array([0]*768*15))
        ct += 1

feats_avg = np.stack(feats_avg)    

feats_tr_avg = feats_avg[tr_idx==1]
feats_te_avg = feats_avg[tr_idx==0]

0it [00:00, ?it/s]

In [9]:
print('feats_tr', feats_tr.shape)  # num_trials * num_latent_feat
# print('feats_te', feats_te.shape)
print('betas_tr', betas_tr.shape)  # num_trials * num_neurons
# print('betas_te', betas_te.shape)

# print('feats_tr_avg', feats_tr_avg.shape)
print('feats_te_avg', feats_te_avg.shape)  # num_avg_trials * num_latent_feat
# print('betas_tr_avg', betas_tr_avg.shape)
print('betas_te_avg', betas_te_avg.shape)  # num_avg_trials * num_neurons


feats_tr (24980, 11520)
betas_tr (24980, 15895)
feats_te_avg (982, 11520)
betas_te_avg (982, 15895)


In [10]:
roi_neuron

{'FFC': 747,
 'FOP1': 208,
 'FOP2': 134,
 'FOP3': 155,
 'FOP4': 410,
 'FOP5': 393,
 'IPS0': 606,
 'IPS1': 462,
 'IPS2': 500,
 'IPS3': 508,
 'IPS4': 65,
 'IPS5': 14,
 'LO1': 343,
 'LO2': 193,
 'LO3': 218,
 'MST': 224,
 'MT': 212,
 'OFC': 1029,
 'OP1': 227,
 'OP2-3': 187,
 'OP4': 523,
 'PF': 1103,
 'PFcm': 299,
 'PFm': 1786,
 'PFop': 454,
 'PHC1': 199,
 'PHC2': 183,
 'RSC': 401,
 'TO1': 284,
 'TO2': 55,
 'V6': 394,
 'V6A': 223,
 'V7': 186,
 'V8': 330,
 'VMV1': 324,
 'VMV2': 245,
 'VMV3': 203,
 'VO1': 216,
 'VO2': 313,
 'VVC': 482,
 'hV4': 473,
 'pOFC': 384}

In [11]:
betas_tr_ = (betas_tr - betas_tr.mean()) / betas_tr.std()
betas_tr_avg_ = (betas_tr_avg - betas_tr_avg.mean()) / betas_tr_avg.std()
betas_te_ = (betas_te - betas_te.mean()) / betas_te.std()
betas_te_avg_ = (betas_te_avg - betas_te_avg.mean()) / betas_te_avg.std()

In [12]:
betas_tr_.shape, betas_te_avg_.shape

((24980, 15895), (982, 15895))

## goal: 0.26555938

In [14]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
alpha = [0.000001,0.00001,0.0001,0.001,0.01, 0.1, 1]

ridge = RidgeCV(alphas=alpha)

preprocess_pipeline = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
    )
pipeline = make_pipeline(
        preprocess_pipeline,
        ridge,
)    

In [17]:
X = betas_tr_
Y = feats_tr
X_te = betas_te_avg_
pipeline.fit(X, Y)
scores = pipeline.predict(X_te)
from sklearn.metrics import mean_squared_error
mean_squared_error(scores,feats_te_avg)

0.753467549664157

## gcn best: 

In [13]:
roi_combined_neuron = {}
neuron = 0

for roi_idx in range(len(roi_neuron)):
    if roi_idx == 1: roi_combined_neuron['FOP'] = 0
    roi_name = list(roi_neuron.keys())[roi_idx]
    if 'FOP' in roi_name:
        if roi_name == 'FOP1':
            roi_combined_neuron['FOP'] = 0
            for name_idx in range(1,6):
                roi_combined_neuron['FOP'] += roi_neuron['FOP'+str(name_idx)] 
    elif 'IPS' in roi_name:
        if roi_name == 'IPS0':
            roi_combined_neuron['IPS01'] = 0
            roi_combined_neuron['IPS25'] = 0
            for name_idx in range(0,6):
                if name_idx <=1: roi_combined_neuron['IPS01'] += roi_neuron['IPS'+str(name_idx)] 
                else: roi_combined_neuron['IPS25'] += roi_neuron['IPS'+str(name_idx)] 

    elif 'LO' in roi_name:
        if roi_name == 'LO1':
            roi_combined_neuron['LO'] = 0
            for name_idx in range(1,4):
                roi_combined_neuron['LO'] += roi_neuron['LO'+str(name_idx)]  

    elif roi_name in ['OP1', 'OP2-3', 'OP4']:
        if roi_name=='OP1': roi_combined_neuron['OP'] = roi_neuron[roi_name]
        else: roi_combined_neuron['OP'] += roi_neuron[roi_name]

    elif 'PHC' in roi_name:
        if roi_name == 'PHC1':
            roi_combined_neuron['PHC'] = 0
            for name_idx in range(1,3):
                roi_combined_neuron['PHC'] += roi_neuron['PHC'+str(name_idx)]  

    elif roi_name in ['TO1', 'TO2']:
        if roi_name=='TO1': roi_combined_neuron['TO'] = roi_neuron[roi_name]
        else: roi_combined_neuron['TO'] += roi_neuron[roi_name]

    elif roi_name in ['V6', 'V6A', 'V7', 'V8']:
        if roi_name=='V6': roi_combined_neuron['V68'] = roi_neuron[roi_name]
        else: roi_combined_neuron['V68'] += roi_neuron[roi_name]
    
    elif roi_name in ['VMV1', 'VMV2', 'VMV3']:
        if roi_name=='VMV1': roi_combined_neuron['VMV'] = roi_neuron[roi_name]
        else: roi_combined_neuron['VMV'] += roi_neuron[roi_name]    

    elif roi_name in ['VO1', 'VO2']:
        if roi_name=='VO1': roi_combined_neuron['VO'] = roi_neuron[roi_name]
        else: roi_combined_neuron['VO'] += roi_neuron[roi_name] 
            
    else:
        roi_combined_neuron[roi_name] = roi_neuron[roi_name]

In [14]:
roi_combined_neuron

{'FFC': 747,
 'FOP': 1300,
 'IPS01': 1068,
 'IPS25': 1087,
 'LO': 754,
 'MST': 224,
 'MT': 212,
 'OFC': 1029,
 'OP': 937,
 'PF': 1103,
 'PFcm': 299,
 'PFm': 1786,
 'PFop': 454,
 'PHC': 382,
 'RSC': 401,
 'TO': 339,
 'V68': 1133,
 'VMV': 772,
 'VO': 529,
 'VVC': 482,
 'hV4': 473,
 'pOFC': 384}

In [15]:
exclude_lst = []
max_neuron_afer_excluding = 0

for idx in range(len(roi_combined_neuron)):
    num_temp = list(roi_combined_neuron.values())[idx] 
    if num_temp > 1300 or num_temp < 400: exclude_lst.append(list(roi_combined_neuron.keys())[idx] )
    else: 
        if num_temp > max_neuron_afer_excluding: max_neuron_afer_excluding = num_temp

# exclude nothing
exclude_lst = []
max_neuron_afer_excluding = max(roi_combined_neuron.values())

exclude_lst , max_neuron_afer_excluding

([], 1786)

In [16]:
roi_combined_neuron

{'FFC': 747,
 'FOP': 1300,
 'IPS01': 1068,
 'IPS25': 1087,
 'LO': 754,
 'MST': 224,
 'MT': 212,
 'OFC': 1029,
 'OP': 937,
 'PF': 1103,
 'PFcm': 299,
 'PFm': 1786,
 'PFop': 454,
 'PHC': 382,
 'RSC': 401,
 'TO': 339,
 'V68': 1133,
 'VMV': 772,
 'VO': 529,
 'VVC': 482,
 'hV4': 473,
 'pOFC': 384}

In [16]:
roi_neuron = roi_combined_neuron
max_num_neurons = max_neuron_afer_excluding
original_betas_tr = []
original_betas_te_avg = []

betas_tr_temp = betas_tr_[:, :list(roi_combined_neuron.values())[0]]
original_betas_tr.append(betas_tr_temp)
zeros = np.zeros((betas_tr_.shape[0], max_num_neurons))
zeros[:, :betas_tr_temp.shape[-1]] = betas_tr_temp
betas_tr_temp = zeros

for roi in range(2, len(list(roi_combined_neuron.keys())[1:]) + 2):
    print(roi, '/', len(list(roi_combined_neuron.keys())[1:])+1, end='\r')
    name = list(roi_combined_neuron.keys())[roi-1]
    if name not in exclude_lst: 
        temp = betas_tr_[:, sum(list(roi_combined_neuron.values())[:roi-1]):sum(list(roi_combined_neuron.values())[:roi])]
        print(name, temp.shape)
        original_betas_tr.append(temp)
        zeros = np.zeros((betas_tr_.shape[0], max_num_neurons))
        zeros[:, :temp.shape[-1]] = temp
        betas_tr_temp = np.hstack([betas_tr_temp, zeros])

betas_te_avg_temp = betas_te_avg_[:, :list(roi_combined_neuron.values())[0]]
original_betas_te_avg.append(betas_te_avg_temp)
zeros = np.zeros((betas_te_avg_.shape[0], max_num_neurons))
zeros[:, :betas_te_avg_temp.shape[-1]] = betas_te_avg_temp
betas_te_avg_temp = zeros
for roi in range(2, len(list(roi_combined_neuron.keys())[1:]) + 2):
    print(roi, '/', len(list(roi_combined_neuron.keys())[1:])+1, end='\r')
    name = list(roi_combined_neuron.keys())[roi-1]
    if name not in exclude_lst: 
        temp = betas_te_avg_[:, sum(list(roi_combined_neuron.values())[:roi-1]):sum(list(roi_combined_neuron.values())[:roi])]
        original_betas_te_avg.append(temp)
        zeros = np.zeros((betas_te_avg_.shape[0], max_num_neurons))
        zeros[:, :temp.shape[-1]] = temp
        betas_te_avg_temp = np.hstack([betas_te_avg_temp, zeros])

betas_tr_ = betas_tr_temp
betas_te_avg_ = betas_te_avg_temp

FOP (24980, 1300)
IPS01 (24980, 1068)
IPS25 (24980, 1087)
LO (24980, 754)
MST (24980, 224)
MT (24980, 212)
OFC (24980, 1029)
OP (24980, 937)
PF (24980, 1103)
PFcm (24980, 299)
PFm (24980, 1786)
PFop (24980, 454)
PHC (24980, 382)
RSC (24980, 401)
TO (24980, 339)
V68 (24980, 1133)
VMV (24980, 772)
VO (24980, 529)
VVC (24980, 482)
hV4 (24980, 473)
pOFC (24980, 384)


In [17]:
betas_tr_.shape, betas_te_avg_.shape

((24980, 39292), (982, 39292))

In [18]:
print(betas_tr_.shape[-1])
print(max_neuron_afer_excluding * (len(roi_neuron) - len(exclude_lst)))

39292
39292


In [19]:
num_roi = len(roi_neuron) - len(exclude_lst)
# num_roi = 4

num_neurons = betas_tr_.shape[-1]
num_pad = (num_neurons // num_roi + 1) * num_roi - num_neurons

betas_tr = np.pad(betas_tr_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_tr_avg = np.pad(betas_tr_avg_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_te = np.pad(betas_te_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_te_avg = np.pad(betas_te_avg_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)

In [20]:
num_neurons_each_roi = int(betas_te_avg.shape[-1]/num_roi)

# conn_measure = connectome.ConnectivityMeasure(kind='correlation')

# function_con = betas_tr.reshape(-1, num_roi, num_neurons_each_roi)

# connectivity = conn_measure.fit_transform([np.mean(function_con, axis=-1)])
# for i in range(len(connectivity[0])):
#     connectivity[0][i, i] = 0  # setting the i-th row to zero

In [22]:
# correlation_matrix = np.zeros((dim, dim))
# correlation_matrix_m1 = np.zeros((dim, dim))
# correlation_matrix_m2 = np.zeros((dim, dim))
# correlation_matrix_p1 = np.zeros((dim, dim))
# correlation_matrix_p2 = np.zeros((dim, dim))
# edge_map = np.zeros((dim, dim))

# for i in range(dim):
#     for j in range(dim):
#         correlation_matrix[i, j] = np.corrcoef([temp_lst[i]], [temp_lst[j]])[0, 1]
#         correlation_matrix_m1[i, j] = np.corrcoef([np.roll(temp_lst[i], shift=-1)], [temp_lst[j]])[0, 1]
#         correlation_matrix_m2[i, j] = np.corrcoef([np.roll(temp_lst[i], shift=-2)], [temp_lst[j]])[0, 1]
#         correlation_matrix_p1[i, j] = np.corrcoef([np.roll(temp_lst[i], shift=1)], [temp_lst[j]])[0, 1]
#         correlation_matrix_p2[i, j] = np.corrcoef([np.roll(temp_lst[i], shift=2)], [temp_lst[j]])[0, 1]
#     correlation_with_shift = np.vstack([correlation_matrix_p2[i], correlation_matrix_p1[i], 
#                                     correlation_matrix[i], 
#                                     correlation_matrix_m1[i], correlation_matrix_m2[i]])
#     edge_map[i][np.where(np.argmax(correlation_with_shift, axis=0)>2)] = 1
#     edge_map[i][np.where(np.argmax(correlation_with_shift, axis=0)<2)] = -1
#     print(np.argmax(correlation_with_shift, axis=0))
        

In [23]:
temp_lst = []
for i in original_betas_tr:
    temp_lst.append(np.mean(i, axis=-1))
temp_lst = np.array(temp_lst)
print(temp_lst.shape)

dim = temp_lst.shape[0]
correlation_matrix = np.zeros((dim, dim))
for i in range(dim):
    for j in range(dim):
        correlation_matrix[i, j] = np.corrcoef([temp_lst[i]], [temp_lst[j]])[0, 1]

connectivity = correlation_matrix.reshape(1, dim, dim)

for i in range(len(connectivity[0])):
    connectivity[0][i, i] = 0  # setting the i-th row to zero

(22, 24980)


In [24]:
connectivity.shape

(1, 22, 22)

In [25]:
%%capture

alpha = 0.05 / dim  

edge_map = np.ones((dim, dim))

for i in range(dim):
    for j in range(dim):
        area0 = np.mean(original_betas_tr[i], axis=1)
        area1 = np.mean(original_betas_tr[j], axis=1)
        area0 = np.diff(area0)[1:]
        area1 = np.diff(area1)[1:]
        # area0 = (np.diff(area0)[1:][1:] - np.diff(area0)[1:][:-1]) / 2
        # area1 = (np.diff(area1)[1:][1:] - np.diff(area1)[1:][:-1]) / 2
    
        data = pd.DataFrame({ 'area0': area0,    'area1': area1 })
        gc_res = grangercausalitytests(data, 1)
        p = gc_res[1][0]['ssr_ftest'][1]
        if p > alpha: edge_map[i, j] = False


In [26]:
adjmatrix = np.where(abs(connectivity[0]) > 0.5)
edge_index = np.array(adjmatrix)  # edge index matrix of shape [2, num_edges]

edge_index.shape

(2, 112)

In [27]:
edge_attr = []
for idx in range(edge_index.shape[1]):
    edge_attr.append(connectivity[0][edge_index[0,idx], edge_index[1,idx]])
edge_attr = np.array(edge_attr)
edge_attr.shape

(112,)

In [28]:
# edge_index

In [29]:
edge_index_temp = []
edge_attr_temp = []
for i in range(edge_attr.shape[0]):
    edge = edge_index.T[i]
    # if edge[0] < edge[1]:
    if edge_map[edge[0], edge[1]] == True:
        edge_index_temp.append(edge)
        edge_attr_temp.append(edge_attr[i])
    else: print('remove edge', edge)
edge_attr = np.array(edge_attr_temp)
edge_index = np.array(edge_index_temp).T

remove edge [0 5]
remove edge [ 2 17]
remove edge [6 0]
remove edge [ 8 10]
remove edge [ 9 10]
remove edge [10  9]
remove edge [12  8]
remove edge [12 10]
remove edge [15  0]
remove edge [15 16]
remove edge [16 18]


In [30]:
edge_index.shape

(2, 101)

In [31]:
cls = feats_tr[0][:768]

In [32]:
num_trial_tr, num_trials_te = feats_tr.shape[0], feats_te_avg.shape[0]

train_dataset = []

for i in range(num_trial_tr):
    train_dataset.append(Data(x=torch.tensor(betas_tr.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_tr[i][768:], dtype=torch.float)
            ))
    
test_dataset = []
for i in range(num_trials_te):
    test_dataset.append(Data(x=torch.tensor(betas_te_avg.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_te_avg[i][768:], dtype=torch.float)
            ))

In [33]:
len(train_dataset), len(test_dataset)

(24980, 982)

In [34]:
train_dataset[0], test_dataset[0]

(Data(x=[22, 1787], edge_index=[2, 101], edge_attr=[101], y=[10752]),
 Data(x=[22, 1787], edge_index=[2, 101], edge_attr=[101], y=[10752]))

In [35]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [36]:
from torch_geometric.nn import GCNConv, TopKPooling, BatchNorm,ChebConv, SAGEConv,GATConv
from torch_geometric.nn import global_mean_pool, global_max_pool,global_add_pool

class GNN(torch.nn.Module):
    def __init__(self, num_neurons_each_roi):
        super(GNN, self).__init__()

        hidden_dim = 512
        
        self.conv1 = ChebConv(num_neurons_each_roi, hidden_dim, K=4)
        self.conv = ChebConv(hidden_dim, hidden_dim, K=3)
        self.bn = BatchNorm(hidden_dim)

        nnconv_channel = 16
        
        self.nnconv = nn.Sequential(
            nn.Conv1d(1, nnconv_channel, kernel_size=1),
            nn.LeakyReLU(),
        )
        self.net = nn.Sequential(
            nn.Conv1d(nnconv_channel, nnconv_channel, kernel_size=3, padding = 1),
            nn.LeakyReLU(),
        )
        self.downsample = nn.Conv1d(nnconv_channel, 1, kernel_size=1)

        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 1024),
            nn.LeakyReLU(),
            nn.Linear(1024, 768*14)
        )

    def forward(self, data):
        x, edge_index, batch,edge_attr = data.x, data.edge_index, data.batch, data.edge_attr
        batch_size = batch.max() + 1

        x = F.relu(self.conv1(x, edge_index, edge_attr))
        
        for i in range(1):
            a = self.conv(x, edge_index, edge_attr)
            a = self.bn(a)
            x = x + F.leaky_relu(a)
        
        x = global_mean_pool(x, batch) # print(x.shape) torch.Size([64, 256])

        x = x.unsqueeze(1)
        x = self.nnconv(x)
        for j in range(2):
            x = x + self.net(x)
        x = self.downsample(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        
        return x

In [37]:
device = 'cuda:1'
model = GNN(num_neurons_each_roi=num_neurons_each_roi).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        batch = batch.to(device)
        batch_num = batch.batch.max() + 1
        feats = batch['y'].reshape(batch_num,-1).to(device)
        pred = model(batch)
        
        loss = loss_fn(pred, feats)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            batch = batch.to(device)
            batch_num = batch.batch.max() + 1
            feats = batch['y'].reshape(batch_num,-1).to(device)
            pred = model(batch)
            loss = loss_fn(pred, feats)
            total_loss_val += loss.item()
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/cs292/text_gcn_roi.pth')
        else:
            ct += 1
            if ct > patience:
                print('end', round(best_loss_val, 5))
                break

[epoch 0] train loss 0.31548 val loss 0.29179
[epoch 1] train loss 0.28341 val loss 0.27408
[epoch 2] train loss 0.26725 val loss 0.26116
[epoch 3] train loss 0.25214 val loss 0.25996
[epoch 4] train loss 0.23761 val loss 0.26074
[epoch 5] train loss 0.22391 val loss 0.2538
[epoch 6] train loss 0.21146 val loss 0.24903
[epoch 7] train loss 0.19977 val loss 0.25
[epoch 8] train loss 0.19014 val loss 0.24659
[epoch 9] train loss 0.18207 val loss 0.24989
[epoch 10] train loss 0.17508 val loss 0.2509
[epoch 11] train loss 0.16841 val loss 0.24897
[epoch 12] train loss 0.16271 val loss 0.25194
[epoch 13] train loss 0.15743 val loss 0.24914
[epoch 14] train loss 0.15267 val loss 0.24908
end 0.24659


In [38]:
device = 'cuda:1'
model = GNN(num_neurons_each_roi=num_neurons_each_roi).to(device)
model.load_state_dict(torch.load('/hdd/yuchen/cs292/text_gcn_roi.pth'))

<All keys matched successfully>

In [39]:
model.eval()
lst = []
lst_gt = []

with torch.no_grad():
    for batch in test_dataloader:
        batch = batch.to(device)
        batch_num = batch.batch.max() + 1
        pred = model(batch)
        batch_num = batch.batch.max() + 1
        feats = batch['y'].reshape(batch_num,-1).to(device)
        
        lst.append(pred.cpu().detach().numpy())
        lst_gt.append(feats.cpu().detach().numpy())

In [40]:
lst = np.vstack(lst)
lst_gt = np.vstack(lst_gt)

In [41]:
from sklearn.metrics import mean_squared_error
mean_squared_error(lst,lst_gt)

0.24607217

In [43]:
lst = lst.reshape(-1, 14, 768)
cls_ = np.tile(cls, (982, 1))
cls_ = np.expand_dims(cls_, 1)
lst = np.concatenate((cls_, lst), axis=1)

In [44]:
lst.shape

(982, 15, 768)

In [45]:
np.save('/hdd/yuchen/cs292/temp_lst_latent_c_roi.npy', lst)

In [10]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
alpha = [0.000001,0.00001,0.0001,0.001,0.01, 0.1, 1]

ridge = RidgeCV(alphas=alpha)

preprocess_pipeline = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
    )
pipeline = make_pipeline(
        preprocess_pipeline,
        ridge,
)    

X = betas_tr
Y = feats_tr
X_te = betas_te_avg
pipeline.fit(X, Y)
scores = pipeline.predict(X_te)
from sklearn.metrics import mean_squared_error
mean_squared_error(scores,feats_te_avg)

0.6971274161290577

In [11]:
num_roi = 4
num_neurons = betas_tr_.shape[-1]
num_pad = (num_neurons // num_roi + 1) * num_roi - num_neurons

betas_tr = np.pad(betas_tr_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_tr_avg = np.pad(betas_tr_avg_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_te = np.pad(betas_te_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_te_avg = np.pad(betas_te_avg_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)

num_neurons_each_roi = int(betas_te_avg.shape[-1]/num_roi)
conn_measure = connectome.ConnectivityMeasure(kind='correlation')
function_con = betas_tr.reshape(-1, num_roi, num_neurons_each_roi)
connectivity = conn_measure.fit_transform([np.mean(function_con, axis=-1)])
for i in range(len(connectivity[0])):
    connectivity[0][i, i] = 0  # Setting the i-th row to zero

/hdd/yuchen/anaconda3/envs/ldm/lib/python3.8/site-packages/nilearn/connectome/connectivity_matrices.py:509: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  covariances_std = [


In [14]:
adjmatrix = np.where(abs(connectivity[0]) > 0.8)
edge_index = np.array(adjmatrix)  # Edge index matrix of shape [2, num_edges]
print(edge_index.shape)

edge_attr = []
for idx in range(edge_index.shape[1]):
    edge_attr.append(connectivity[0][edge_index[0,idx], edge_index[1,idx]])
edge_attr = np.array(edge_attr)

(2, 4)


In [15]:
num_trial_tr, num_trials_te = feats_tr.shape[0], feats_te_avg.shape[0]

train_dataset = []
for i in range(num_trial_tr):
    train_dataset.append(Data(x=torch.tensor(betas_tr.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_tr[i], dtype=torch.float)
            ))
    
test_dataset = []
for i in range(num_trials_te):
    test_dataset.append(Data(x=torch.tensor(betas_te_avg.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_te_avg[i], dtype=torch.float)
            ))

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [16]:
from torch_geometric.nn import GCNConv, TopKPooling, BatchNorm,ChebConv, SAGEConv,GATConv
from torch_geometric.nn import global_mean_pool, global_max_pool,global_add_pool

class GCNWithPooling(torch.nn.Module):
    def __init__(self, num_neurons_each_roi):
        super(GCNWithPooling, self).__init__()
        
        self.conv1 = ChebConv(num_neurons_each_roi, 256, K=4)

        self.dropout = nn.Dropout(p=0.1)
        self.conv = ChebConv(256, 256, K=3)
        self.bn = BatchNorm(256)

        nnconv_channel = 16
        self.nnconv = nn.Sequential(
            nn.Conv1d(1, nnconv_channel, kernel_size=1),
            nn.ReLU(), 
        )
        self.net = nn.Sequential(
            nn.Conv1d(nnconv_channel, nnconv_channel, kernel_size=3, padding = 1),
            nn.ReLU(),
        )
        self.downsample = nn.Conv1d(nnconv_channel, 1, kernel_size=1)

        self.fc = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 6400)
        )

    def forward(self, data):
        x, edge_index, batch,edge_attr = data.x, data.edge_index, data.batch, data.edge_attr
        batch_size = batch.max() + 1

        x = F.relu(self.conv1(x, edge_index, edge_attr))

        for i in range(1):
            a = self.conv(x, edge_index, edge_attr)
            a = self.bn(a)
            x = x + F.relu(a)

        x = global_mean_pool(x, batch) # print(x.shape) torch.Size([64, 256])

        x = x.unsqueeze(1)
        x = self.nnconv(x)
        
        for j in range(2):
            x = x + self.net(x)
        x = self.downsample(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        
        return x

In [19]:
device = 'cuda:1'
model = GCNWithPooling(num_neurons_each_roi=num_neurons_each_roi).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        batch = batch.to(device)
        batch_num = batch.batch.max() + 1
        feats = batch['y'].reshape(batch_num,-1).to(device)
        pred = model(batch)
        
        loss = loss_fn(pred, feats)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            batch = batch.to(device)
            batch_num = batch.batch.max() + 1
            feats = batch['y'].reshape(batch_num,-1).to(device)
            pred = model(batch)
            loss = loss_fn(pred, feats)
            total_loss_val += loss.item()
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/modelweights_visual_gcn_test.pth')
        else:
            ct += 1
            if ct > patience:
                print('end')
                break

[epoch 0] train loss 0.65145 val loss 0.62354
[epoch 1] train loss 0.64557 val loss 0.62117
[epoch 2] train loss 0.64482 val loss 0.62083
[epoch 3] train loss 0.64456 val loss 0.62064
[epoch 4] train loss 0.64436 val loss 0.62031
[epoch 5] train loss 0.64402 val loss 0.61988
[epoch 6] train loss 0.64311 val loss 0.61861
[epoch 7] train loss 0.64154 val loss 0.6169
[epoch 8] train loss 0.63933 val loss 0.61434
[epoch 9] train loss 0.63718 val loss 0.61329
[epoch 10] train loss 0.63523 val loss 0.61178
[epoch 11] train loss 0.63353 val loss 0.61143
[epoch 12] train loss 0.63188 val loss 0.61121
[epoch 13] train loss 0.63019 val loss 0.61096
[epoch 14] train loss 0.62867 val loss 0.61115
[epoch 15] train loss 0.62739 val loss 0.611
[epoch 16] train loss 0.62583 val loss 0.6112
[epoch 17] train loss 0.62411 val loss 0.61157
[epoch 18] train loss 0.6224 val loss 0.61154
[epoch 19] train loss 0.62063 val loss 0.61303
end


In [181]:
out = model(batch)
out.shape

torch.Size([16, 6400])

In [151]:
model(dataset[0]).shape

torch.Size([1480, 64])

In [146]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [169]:
out = model(batch)
out.reshape(16, -1, 64).shape

torch.Size([16, 1480, 64])

In [140]:
train_loader = DataLoader(dataset, batch_size=1)
len(train_loader)

100

In [148]:
from torch_geometric.loader import DataLoader

In [53]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# Load a dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

In [54]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [55]:
data.num_classes

AttributeError: 'GlobalStorage' object has no attribute 'num_classes'